# DLNLP W4 Notebook 2
## Two-layer neural network for classification

In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline

First, we generate a simple data set for binary classification.

In [ ]:
X_train = []
t_train = []

for mu, c in [([1.,1.], 1), ([-1., -1.], 1), ([1., -1.], 0), ([-1., 1.], 2)]:
    X_train.append(np.random.multivariate_normal(np.array(mu), .2*np.eye(2), 50))
    t_train.append(np.ones(50)*c)
    
X_train = np.concatenate(X_train)
t_train = np.concatenate(t_train)

fig, ax = plt.subplots()
ax.scatter(X_train[t_train==0, 0], X_train[t_train==0, 1], label="Class 0")
ax.scatter(X_train[t_train==1, 0], X_train[t_train==1, 1], label="Class 1")
ax.scatter(X_train[t_train==2, 0], X_train[t_train==2, 1], label="Class 2")
ax.set_xlabel(r'$X_train_1$')
ax.set_ylabel(r'$X_train_2$')
ax.legend()

# Conver t_train to 1-of-K
t_train = np.eye(3)[t_train.astype(int)]

Then we define the graph for a two-layer neural network.

In [ ]:
# Input and output
x = tf.placeholder(tf.float32, [None, 2])
t = tf.placeholder(tf.float32, [None, 3])

# Defined the model parameters
W1 = tf.get_variable("W1", [2, 5], initializer=tf.random_normal_initializer)
b1 = tf.get_variable("b1", [5], initializer=tf.random_normal_initializer)
W2 = tf.get_variable("W2", [5, 3], initializer=tf.random_normal_initializer)
b2 = tf.get_variable("b2", [3], initializer=tf.random_normal_initializer)
    
# Construct model
a1 = tf.matmul(x, W1) + b1
z1 = tf.nn.tanh(a1)
a2 = tf.matmul(z1, W2) + b2
y = tf.nn.softmax(a2)

# Variables for prediction and accuracy
prediction = tf.argmax(y, 1)
accuracy = tf.reduce_mean(tf.cast(tf.equal(prediction, tf.argmax(t, 1)), tf.float32))

# Difine the loss function
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=t, logits=a2))

# Define the optimizer operation
learning_rate = tf.placeholder(tf.float32)
optimizer = tf.train.GradientDescentOptimizer(learning_rate = learning_rate).minimize(loss)

# Make an operation that initializes the variables
init = tf.global_variables_initializer()

Finally we run a session to train the network and evaluate the accoracy on the training data.

In [ ]:
x_test = np.linspace(0, 1, 100)[:, np.newaxis]
y_value_list = []

# Start a new session
with tf.Session() as session:
    # Initialize the values
    session.run(init)
    
    # Training cycle
    for epoch in range(10000):
        _, loss_value = session.run([optimizer, loss], feed_dict={x: X_train, t: t_train, learning_rate: 0.01})
        
        if epoch % 1000 == 0:
            print("Epoch: ", epoch, "loss =", loss_value)            
            
    print("Optimization done")

    # Evaluate the accuracy on the test set
    accuracy_value, y_train = session.run([accuracy, y], feed_dict={x: X_train, t: t_train})
    print("Accuracy:", accuracy_value)

In [ ]:
fig, ax = plt.subplots()

correct_mask = np.argmax(t_train, axis=1) == np.argmax(y_train, axis=1)
incorrect_mask = np.logical_not(correct_mask)

ax.scatter(X_train[correct_mask, 0], X_train[correct_mask, 1], color='C0', label="Correct")
ax.scatter(X_train[incorrect_mask, 0], X_train[incorrect_mask, 1], color='C3', label="Incorrect")

ax.set_xlabel(r'$X_train_1$')
ax.set_ylabel(r'$X_train_2$')
ax.legend()
plt.show()